In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import concurrent.futures

## Define the function to scrape a page
def scrape_page(page_link):
    r = requests.get(page_link, headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    entries_block = soup.find("ul", {"id": "entry-item-list"})
    
    if not entries_block:
        print(f"No entries found on page: {page_link}")
        return []
    
    entries_block = entries_block.find_all("li")
    page_entries = []
    for entry in entries_block:
        username = entry.get("data-author")
        content = entry.find("div", {"class": "content"}).text.strip()  # Strip leading/trailing whitespaces
        date = entry.find("a", {"class": "entry-date permalink"}).text
        page_entries.append((username, content, date))
    
    return page_entries

if __name__ == "__main__":
    mainLink = input("Please input your URL")  # Get the link from the user
    mainLink = mainLink.split("?")[0]  # If the link has a query string, remove it
    
    headers = {
        "user-agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36 OPR/84.0.4316.31")
    }

    # Get the page count from the "data-pagecount" attribute
    r = requests.get(mainLink, headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    page_count_div = soup.find("div", {"class": "pager"})
    
    if not page_count_div:
        raise ValueError("Could not find the 'pager' div on the page.")
    
    pageCount = int(page_count_div.get("data-pagecount"))

    # Create an empty list to store all entries
    all_entries = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        page_links = [mainLink + "?p=" + str(i) for i in range(1, pageCount + 1)]
        results = executor.map(scrape_page, page_links)
        for result in results:
            all_entries.extend(result)

    # Create DataFrame from scraped data
    EntryDF = pd.DataFrame(all_entries, columns=["username", "entry", "date"])
    
    # Remove rows with missing or irrelevant data
    EntryDF.dropna(inplace=True)
    
    # Save DataFrame to CSV with proper alignment
    EntryDF.to_csv("Entries.csv", encoding="utf-8-sig", index_label="Index")
    print("Entries successfully retrieved!")

